In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
from datetime import date
import datetime
from selenium.webdriver.support.ui import Select
import time
from selenium.common.exceptions import NoSuchElementException



In [2]:
def time_to_seconds(t):
    minutes, seconds, milliseconds = map(int, t.replace('.', ':').split(':'))
    return minutes * 60 + seconds + milliseconds / 1000

def append_data_and_update_csv(data):
    global df
    # Create a DataFrame from the new data
    new_data_df = pd.DataFrame(data)
    # Append new data to the dataframe
    df = pd.concat([df, new_data_df], ignore_index=True)
    # Update the CSV file
    df.to_csv('live_dat.csv', index=False)
laptime_soft, laptime_medium, laptime_hard, lap_no, driver_name, gap_time, airtemp_arr, tracktemp_arr, air_pressure_arr, humidity_arr, wind_speed_arr, direction_arr, speed_arr = ([] for _ in range(13))


def get_weather_data():
    airtemp=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_airtemp"]')))[0].text
    airtemp_arr.append(airtemp)
    tracktemp=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_tracktemp"]')))[0].text
    tracktemp_arr.append(tracktemp)
    air_pressure=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_pressure"]')))[0].text
    air_pressure_arr.append(air_pressure)
    humidity=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_humidity"]')))[0].text
    humidity_arr.append(humidity)
    wind_speed=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_humidity"]')))[0].text
    wind_speed_arr.append(wind_speed)
    direction=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_winddirection"]')))[0].text
    direction_arr.append(direction) 

In [5]:
driver=webdriver.Edge()

driver.get('https://f1.tfeed.net/2024/saudi_arabia')
driver.implicitly_wait(5)
element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="uLogin_2"]/div/div[9]')))
element.click()
#dropdown_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="replay_laps_select"]')))

# Initialize Select object with the dropdown element
#dropdown = Select(dropdown_element)

try:
    drop=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="replay_laps_select"]/option[2]')))[0]
    drop.click()
    # Try to select an option by its value
    #dropdown.select_by_value('0')
except NoSuchElementException:
    # If the option is not found, refresh the page

    driver.refresh()
    drop.click()
    #dropdown.select_by_value('0')

compound_button=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="header_stats_seq_id"]')))[0]


cols=['lap_no', 'laptime_soft', 'laptime_medium', 'laptime_hard', 'Driver Name', 'gap_time', 'airtemp', 'tracktemp', 'Air Pressure', 'Humidity%', 'wind Speed', 'Wind Direction', 'Max speed']

df=pd.DataFrame(columns=cols)



#time.sleep(20)


time.sleep(10)
total_lap = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="stats_si_laps"]')))[0].text
total_lap = total_lap.split('/')[0]
total_lap = int(total_lap)
print(total_lap)
for j in range (4,total_lap):
    formatted_number_lap = '{:02}'.format(j)
    drop=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="replay_laps_select"]/option[{formatted_number_lap}]')))[0]
    drop.click()
    time.sleep(20) 
    for i in range(1,21):
        formatted_number = '{:02}'.format(i)
        laptime_time = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_t"]')))[0].text
        print(laptime_time)
        if laptime_time == '':
        # If it's an empty string, set laptime to a default value
            laptime = 0
        else:
            laptime = time_to_seconds(laptime_time)
        #laptime = time_to_seconds(laptime_time)
        driver_text=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_nick"]')))[0].text
        driver_name.append(driver_text)
        try:
            gap=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_gap"]')))[0].text
            gap_time.append(gap)
        except NoSuchElementException:
            gap_time.append('0')
        get_weather_data()
        lap = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_lap"]')))[0].text
        print(lap)
        val=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"span[class='stats_header stats_header_small stats_tyres']")))[0].text
        while val != 'TYRES':
            compound_button.click()
            val=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"span[class='stats_header stats_header_small stats_tyres']")))[0].text
            print(val)
            time.sleep(.3)
        
        tyre = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_tyres"]')))[0].text#split tyre
        print(tyre)
        
        while val != 'M. SPD':
            compound_button.click()
            val=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"span[class='stats_header stats_header_small stats_tyres']")))[0].text
            print(val)
            time.sleep(.3)
        speed=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_mspeed_a"]')))[0].text
        speed_arr.append(speed)
        lap_no.append(lap)
        if tyre == '' :
            laptime_soft.append('NULL')
            laptime_medium.append('NULL')
            laptime_hard.append('NULL')
        else:
            length=len(tyre)
            parts=list(tyre)
            for i in range(0,length):
                if parts[i]=='S' and parts[i+1].isdigit():
                    laptime_soft.append(laptime)
                    laptime_medium.append('NULL')
                    laptime_hard.append('NULL')
                elif parts[i]=='M' and parts[i+1].isdigit():
                    laptime_soft.append('NULL')
                    laptime_medium.append(laptime)
                    laptime_hard.append('NULL')
                elif parts[i]=='H' and parts[i+1].isdigit():
                    laptime_soft.append('NULL')
                    laptime_medium.append('NULL')
                    laptime_hard.append(laptime)
            
        new_data = {'lap_no': lap_no,
                    'laptime_soft': laptime_soft,
                    'laptime_medium': laptime_medium,
                    'laptime_hard': laptime_hard,
                    'Driver Name': driver_name,
                    'gap_time': gap_time,
                    'airtemp': airtemp_arr,
                    'tracktemp': tracktemp_arr,
                    'Air Pressure': air_pressure_arr,
                    'Humidity%': humidity_arr,
                    'wind Speed': wind_speed_arr,
                    'Wind Direction': direction_arr,
                    'Max speed': speed_arr}
        append_data_and_update_csv(new_data)
        [ls.clear() for ls in [laptime_soft, laptime_medium, laptime_hard, lap_no, driver_name, gap_time, airtemp_arr, tracktemp_arr, air_pressure_arr, humidity_arr, wind_speed_arr, direction_arr, speed_arr]]

        


50
1:34.205


NameError: name 'driver_name' is not defined

In [37]:
tyre

''

In [6]:
[ls.clear() for ls in [laptime_soft, laptime_medium, laptime_hard, lap_no, driver_name, gap_time, airtemp_arr, tracktemp_arr, air_pressure_arr, humidity_arr, wind_speed_arr, direction_arr, speed_arr]]


NameError: name 'laptime_soft' is not defined

In [8]:


laptime_soft, laptime_medium, laptime_hard, lap_no, driver_name, gap_time, airtemp_arr, tracktemp_arr, air_pressure_arr, humidity_arr, wind_speed_arr, direction_arr, speed_arr = ([] for _ in range(13))
val=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"span[class='stats_header stats_header_small stats_tyres']")))[0].text
    

def get_weather_data():
    airtemp=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_airtemp"]')))[0].text
    airtemp_arr.append(airtemp)
    tracktemp=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_tracktemp"]')))[0].text
    tracktemp_arr.append(tracktemp)
    air_pressure=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_pressure"]')))[0].text
    air_pressure_arr.append(air_pressure)
    humidity=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_humidity"]')))[0].text
    humidity_arr.append(humidity)
    wind_speed=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_humidity"]')))[0].text
    wind_speed_arr.append(wind_speed)
    direction=WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="weather_winddirection"]')))[0].text
    direction_arr.append(direction) 







total_lap = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="stats_si_laps"]')))[0].text
total_lap = total_lap.split('/')[0]
total_lap = int(total_lap)
print(total_lap)
for j in range (4,total_lap):
    formatted_number_lap = '{:02}'.format(j)
    drop=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="replay_laps_select"]/option[{formatted_number_lap}]')))[0]
    drop.click()
    time.sleep(20) 
    for i in range(1,21):
        formatted_number = '{:02}'.format(i)
        laptime_time = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_t"]')))[0].text
        print(laptime_time)
        if laptime_time == '':
        # If it's an empty string, set laptime to a default value
            laptime = 0
        else:
            laptime = time_to_seconds(laptime_time)
        #laptime = time_to_seconds(laptime_time)
        driver_text=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_nick"]')))[0].text
        driver_name.append(driver_text)
        try:
            gap=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_gap"]')))[0].text
            gap_time.append(gap)
        except NoSuchElementException:
            gap_time.append('0')
        get_weather_data()
        lap = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_lap"]')))[0].text
        print(lap)
        while val != 'TYRES':
            compound_button.click()
            val=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"span[class='stats_header stats_header_small stats_tyres']")))[0].text
            print(val)
            time.sleep(.3)
        
        tyre = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_tyres"]')))[0].text#split tyre
        print(tyre)
        
        
        while val != 'M. SPD':
            compound_button.click()
            val=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"span[class='stats_header stats_header_small stats_tyres']")))[0].text
            print(val)
            time.sleep(.3)
        speed=WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="i_{formatted_number}_mspeed_a"]')))[0].text
        speed_arr.append(speed)
        print(laptime)
        lap_no.append(lap)
        if tyre == '' :
            laptime_soft.append('NULL')
            laptime_medium.append('NULL')
            laptime_hard.append('NULL')
        else:
            length=len(tyre)
            parts=list(tyre)
            for i in range(0,length):
                if parts[i]=='S' and parts[i+1].isdigit():
                    laptime_soft.append(laptime)
                    laptime_medium.append('NULL')
                    laptime_hard.append('NULL')
                elif parts[i]=='M' and parts[i+1].isdigit():
                    laptime_soft.append('NULL')
                    laptime_medium.append(laptime)
                    laptime_hard.append('NULL')
                elif parts[i]=='H' and parts[i+1].isdigit():
                    laptime_soft.append('NULL')
                    laptime_medium.append('NULL')
                    laptime_hard.append(laptime)
            
            
        new_data = {'lap_no': lap_no,
                    'laptime_soft': laptime_soft,
                    'laptime_medium': laptime_medium,
                    'laptime_hard': laptime_hard,
                    'Driver Name': driver_name,
                    'gap_time': gap_time,
                    'airtemp': airtemp_arr,
                    'tracktemp': tracktemp_arr,
                    'Air Pressure': air_pressure_arr,
                    'Humidity%': humidity_arr,
                    'wind Speed': wind_speed_arr,
                    'Wind Direction': direction_arr,
                    'Max speed': speed_arr}
        append_data_and_update_csv(new_data)
        [ls.clear() for ls in [laptime_soft, laptime_medium, laptime_hard, lap_no, driver_name, gap_time, airtemp_arr, tracktemp_arr, air_pressure_arr, humidity_arr, wind_speed_arr, direction_arr, speed_arr]]

        

47
1:34.205
3
POSITION
TLMY.
M. SPD
SPD. T.
POINTS
BEST LAP
BST SECT.
AUTO
TYRES
M2
INTERVAL
GAP
PARTNER
POSITION
TLMY.
M. SPD
94.205
1:34.179
3
SPD. T.
POINTS
BEST LAP
BST SECT.
AUTO
TYRES
M2
INTERVAL
GAP
PARTNER
POSITION
TLMY.
M. SPD
94.179
1:34.515
3
SPD. T.
POINTS
BEST LAP
BST SECT.
AUTO
TYRES
M2
INTERVAL
GAP
PARTNER
POSITION
TLMY.
M. SPD
94.515
1:35.082
3
SPD. T.
POINTS
BEST LAP
BST SECT.
AUTO
TYRES
M2
INTERVAL
GAP
PARTNER
POSITION
TLMY.
M. SPD
95.082
1:35.724
3
SPD. T.
POINTS
BEST LAP
BST SECT.
AUTO
TYRES

INTERVAL
GAP
PARTNER
POSITION
TLMY.
M. SPD
95.724
1:35.246
3
SPD. T.
POINTS
BEST LAP
BST SECT.
AUTO
TYRES

INTERVAL
GAP
PARTNER
POSITION
TLMY.
M. SPD
95.246
1:35.146
3
SPD. T.
POINTS
BEST LAP
BST SECT.
AUTO
TYRES

INTERVAL
GAP
PARTNER
POSITION
TLMY.
M. SPD
95.146
1:35.541
3
SPD. T.
POINTS
BEST LAP
BST SECT.
AUTO
TYRES

INTERVAL
GAP
PARTNER
POSITION
TLMY.
M. SPD
95.541
1:36.094
3
SPD. T.
POINTS
BEST LAP
BST SECT.
AUTO
TYRES
M3
INTERVAL
GAP
PARTNER
POSITION
TLMY.
M. SPD
96.094
1:

PermissionError: [Errno 13] Permission denied: 'live_dat.csv'

In [66]:
lap_no

[]

In [5]:
df = pd.DataFrame({
    'lap':lap_no,
    'laptime_soft': laptime_soft,
    'laptime_medium': laptime_medium,
    'laptime_hard': laptime_hard
})

df

NameError: name 'lap' is not defined

In [94]:
df.to_csv('laptime.csv',index=False)